In [22]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np
import re

In [3]:
inflow = pd.read_parquet('/uss/hdsi-prismdata/q1-ucsd-inflows.pqt')
outflow = pd.read_parquet('/uss/hdsi-prismdata/q1-ucsd-outflows.pqt')

In [26]:
memos = pd.Series(outflow[outflow['category'] != outflow['memo']]['memo'].unique()).sort_values()
memos

295465    # (800) 544-XXXX 139 EAST FOURTH STREET CINCIN...
83894                                                 # 101
83885                                                 # 102
83878                                                 # 103
83883                                                 # 104
                                ...                        
13483                                                zpizza
311758                 zulily CC zulilyEPAY ***********XXXX
31255     zulily CC zulilyEPAY XXXXXX XXXXXXXXX XXXXXXXX...
31245     zulily CC zulilyEPAY XXXXXX XXXXXXXXXX XXXXXXX...
116902      zulilyEPAY zulily CC XXXXXXXXX INTERNET PAYMENT
Length: 528766, dtype: object

In [27]:
np.savetxt('../memos.txt', memos.values, fmt='%s')

In [5]:
# pattern = r"X{2,}"
# matches = [s for s in memos if re.search(pattern, s)]
# matches

In [6]:
memos_clean = memos.str.lower()
memos_clean = memos_clean.apply(lambda x: re.sub(r'[^a-z0-9\s]', '', x)) # Remove special characters
memos_clean = memos_clean.apply(lambda x: re.sub(r'\s+', ' ', x).strip()) # Remove whitespace
print(memos)
print(memos_clean)

0                 TST* Casa Del Rio - Exp Fairlawn OH 09/24
1                                        Buffalo Wild Wings
2                                           Oculus CA 04/16
3                               LOS GIRASOLES STOW OH 03/08
4                                 BUZZIS LAUNDRY 1 OH 03/28
                                ...                        
528761    DEBIT CARD WITHDRAWAL PURCHASEAudible*Z295DXXX...
528762    DEBIT CARD WITHDRAWAL PURCHASEJACK IN THE BOX ...
528763    DEBIT CARD WITHDRAWAL PURCHASEAmazon Prime*TI4...
528764    POS WITHDRAWALAZ LOT QUIKTRIP XXXX XXXX E INDI...
528765    POS WITHDRAWALWAL-MART #XXXX XXXX E MCKELLIPS ...
Length: 528766, dtype: object
0                     tst casa del rio exp fairlawn oh 0924
1                                        buffalo wild wings
2                                            oculus ca 0416
3                                los girasoles stow oh 0308
4                                  buzzis laundry 1 oh 0328
          

In [7]:
vectorizer = CountVectorizer(ngram_range=(5, 5), min_df=10)
X = vectorizer.fit_transform(memos_clean)

term_counts = X.sum(axis=0)
frequencies = [(term, term_counts[0, idx]) for term, idx in vectorizer.vocabulary_.items()]

sorted_frequencies = sorted(frequencies, key=lambda x: x[1], reverse=True)


AttributeError: 'list' object has no attribute 'size'

In [8]:
len(sorted_frequencies)

18873

In [ ]:
for term, freq in sorted_frequencies[:10]:
    print(f'"{term}": {freq}')

In [ ]:
memos_clean[memos_clean.str.contains('xxxxxxxxxx ca sxxxxxxxxxxxxxxx card xxxx')]

In [160]:
memos_clean[memos_clean.str.lower().str.contains('visa check card')]

7828      pos debit visa check card xxxx amazoncom1x3bt3...
7830      pos debit visa check card xxxx instacart159 ht...
7834      pos debit visa check card xxxx amazoncom1r2gz0...
7839      pos debit visa check card xxxx amazoncom6b0ir4...
7841      pos debit visa check card xxxx amazoncompg26a1...
                                ...                        
526419    pos debit visa check card xxxx mcdonalds fxxxx...
526420    pos debit visa check card xxxx fastfix jewelry...
526421    pos debit visa check card xxxx lowes xxxx fair...
526422    pos debit visa check card xxxx target t xxxx h...
526423    pos debit visa check card xxxx sams club vacav...
Length: 11667, dtype: object

In [62]:
# for memo in pd.Series(memos).str.split():
#     if '/' in memo[0]:
#       n  print(memo)

In [13]:
sample = memos.sample(10_000)